In [1]:
import dew_gwdata as gd

In [2]:
db = gd.sageodata()

In [5]:
# 280002 49254 32458
print(db.hydrostrat_logs([32458]).pipe(gd.cleanup_columns))

    well_id  unit_depth_from  unit_depth_to unit_code  \
0  6532-714            13.11          19.20  Qpap(Q1)   
1  6532-714            23.16          23.47  Qpap(Q2)   
2  6532-714            27.43          32.61  Qpap(Q3)   
3  6532-714            49.99          57.91  Qpap(Q5)   
4  6532-714            63.40          82.91  Qpap(Q6)   
5  6532-714            82.91         112.78     TpQag   

                                   unit_desc  hyd_int_depth_from  \
0  Pooraka Formation (Quaternary sand layer)                0.00   
1  Pooraka Formation (Quaternary sand layer)                0.00   
2  Pooraka Formation (Quaternary sand layer)                0.00   
3  Pooraka Formation (Quaternary sand layer)                0.00   
4  Pooraka Formation (Quaternary sand layer)                0.00   
5                                Gibbon beds               82.91   

   hyd_int_depth_to hyd_int_depth_to_greater  \
0             82.91                     None   
1             82.91        

In [5]:
def hydrostrat_logs_simple(self, DH_NO):
    hs = self.hydrostrat_logs(DH_NO)
    return hs

In [9]:
df = hydrostrat_logs_simple(db, db.find_wells("CAR081")).pipe(gd.cleanup_columns)

In [ ]:
df["depth_from"] = df.unit_depth_from
df["depth_to"] = df.unit_depth_to
df.loc[~df.subunit_depth_from.isnull()] = df.

In [10]:
rows = []
for idx, row in df.iterrows()

,well_id,unit_depth_from,unit_depth_to,hydro_type,hydro_depth_to_greater_flag,comments,map_symbol,strat_name,hydrostrat_desc,subunit_depth_from,subunit_depth_to,hydro_subunit_code,hydro_subunit_desc
0,CAR081,0.0,4.0,None,None,None,Qpcb,Bridgewater Formation,Bridgewater - Unconfined,NaN,NaN,None,None
1,CAR081,4.0,150.0,None,None,None,Thgr,Green Point Member,None,4.0,38.0,U1,"unit 1, topmost aquifer"
2,CAR081,4.0,150.0,None,None,None,Thgr,Green Point Member,None,38.0,98.0,U2,"unit 2, aquitard"
3,CAR081,4.0,150.0,None,None,None,Thgr,Green Point Member,None,98.0,120.0,U3,"unit 3, middle aquifer"
4,CAR081,4.0,150.0,None,None,None,Thgr,Green Point Member,None,120.0,150.0,U4,"unit 4, aquitard"
5,CAR081,150.0,170.0,None,None,None,Thgc,Camelback Member,None,NaN,NaN,None,None


In [56]:
sql = """
 SELECT          dh.drillhole_no AS dh_no,
                dh.unit_no      AS unit_long,
                To_char(dh.map_100000_no)
                                || '-'
                                || To_char(dh.dh_seq_no) AS unit_hyphen,
                Trim(To_char(dh.obs_well_plan_code))
                                || Trim(To_char(dh.obs_well_seq_no, '000')) AS obs_no,
                dh.dh_name,
                dh.amg_easting          AS easting,
                dh.amg_northing         AS northing,
                dh.amg_zone             AS zone,
                dh.neg_lat_deg_real     AS latitude,
                dh.long_deg_real        AS longitude,
                summ.aq_subaq           AS aquifer,
                hs_int.hydro_depth_from AS unit_depth_from,
                hs_int.hydro_depth_to   AS unit_depth_to,
                hs_int.hydro_type,
                hs_int.hydro_depth_to_greater_flag,
                hs_int.comments,
                s_unit.map_symbol,
                s_unit.strat_name,
                s_unit.hydrostrat_desc,
                hs_subint.hydro_depth_from AS subunit_depth_from,
                hs_subint.hydro_depth_to   AS subunit_depth_to,
                hs_subint.hydro_subunit_code,
                hs_subunit.hydro_subunit_desc,
                hs_int.hydro_depth_from as depth_from,
                hs_int.hydro_depth_to as depth_to,
                Trim(To_char(s_unit.map_symbol)) as hsunit_symbol,
                Trim(To_char(s_unit.strat_name)) as hsunit_desc
FROM            dhdb.wa_hydrostrat_int_vw hs_int
left outer join dhdb.st_strat_unit_vw s_unit
ON              hs_int.strat_unit_no = s_unit.strat_unit_no
left outer join dhdb.wa_hydrostrat_subint_vw hs_subint
ON              hs_int.hydro_int_no = hs_subint.hydro_int_no
left outer join dhdb.wa_hydrostrat_subunit_vw hs_subunit
ON              hs_subint.strat_unit_no = hs_subunit.strat_unit_no
AND             hs_subint.hydro_subunit_code = hs_subunit.hydro_subunit_code
left outer join dhdb.dd_drillhole_vw dh
ON              hs_int.drillhole_no = dh.drillhole_no
left outer join dhdb.dd_drillhole_summary_vw summ
ON              hs_int.drillhole_no = summ.drillhole_no
WHERE           hs_int.drillhole_no IN (280002)
and hs_subint.hydro_subunit_code is null

union all

SELECT          dh.drillhole_no AS dh_no,
                dh.unit_no      AS unit_long,
                To_char(dh.map_100000_no)
                                || '-'
                                || To_char(dh.dh_seq_no) AS unit_hyphen,
                Trim(To_char(dh.obs_well_plan_code))
                                || Trim(To_char(dh.obs_well_seq_no, '000')) AS obs_no,
                dh.dh_name,
                dh.amg_easting          AS easting,
                dh.amg_northing         AS northing,
                dh.amg_zone             AS zone,
                dh.neg_lat_deg_real     AS latitude,
                dh.long_deg_real        AS longitude,
                summ.aq_subaq           AS aquifer,
                hs_int.hydro_depth_from AS unit_depth_from,
                hs_int.hydro_depth_to   AS unit_depth_to,
                hs_int.hydro_type,
                hs_int.hydro_depth_to_greater_flag,
                hs_int.comments,
                s_unit.map_symbol,
                s_unit.strat_name,
                s_unit.hydrostrat_desc,
                hs_subint.hydro_depth_from AS subunit_depth_from,
                hs_subint.hydro_depth_to   AS subunit_depth_to,
                hs_subint.hydro_subunit_code,
                hs_subunit.hydro_subunit_desc,
                hs_subint.hydro_depth_from as depth_from,
                hs_subint.hydro_depth_to as depth_to,
                To_char(s_unit.map_symbol) 
                || '(' 
                || 
                To_char(hs_subint.hydro_subunit_code) || ')' as hsunit_symbol,
                Trim(To_char(s_unit.strat_name)) || ' (' || Trim(To_char(hs_subunit.hydro_subunit_desc)) || ')' as hsunit_desc
FROM            dhdb.wa_hydrostrat_int_vw hs_int
left outer join dhdb.st_strat_unit_vw s_unit
ON              hs_int.strat_unit_no = s_unit.strat_unit_no
left outer join dhdb.wa_hydrostrat_subint_vw hs_subint
ON              hs_int.hydro_int_no = hs_subint.hydro_int_no
left outer join dhdb.wa_hydrostrat_subunit_vw hs_subunit
ON              hs_subint.strat_unit_no = hs_subunit.strat_unit_no
AND             hs_subint.hydro_subunit_code = hs_subunit.hydro_subunit_code
left outer join dhdb.dd_drillhole_vw dh
ON              hs_int.drillhole_no = dh.drillhole_no
left outer join dhdb.dd_drillhole_summary_vw summ
ON              hs_int.drillhole_no = summ.drillhole_no
WHERE           hs_int.drillhole_no IN (280002)
 and hs_subint.hydro_subunit_code is not null

order by unit_depth_from, subunit_depth_from
"""

In [57]:
print(
    db.query(sql)
    .pipe(gd.cleanup_columns)
    .drop(["hydro_depth_to_greater_flag", "comments", "hydro_type"], axis=1)
)

  well_id  unit_depth_from  unit_depth_to map_symbol             strat_name  \
0  CAR081              0.0            4.0       Qpcb  Bridgewater Formation   
1  CAR081              4.0          150.0       Thgr     Green Point Member   
2  CAR081              4.0          150.0       Thgr     Green Point Member   
3  CAR081              4.0          150.0       Thgr     Green Point Member   
4  CAR081              4.0          150.0       Thgr     Green Point Member   
5  CAR081            150.0          170.0       Thgc       Camelback Member   

            hydrostrat_desc  subunit_depth_from  subunit_depth_to  \
0  Bridgewater - Unconfined                 NaN               NaN   
1                      None                 4.0              38.0   
2                      None                38.0              98.0   
3                      None                98.0             120.0   
4                      None               120.0             150.0   
5                      None     

In [ ]:
sql2 = """
SELECT          
                hs_int.hydro_depth_from AS unit_depth_from,
                hs_int.hydro_depth_to   AS unit_depth_to,
                hs_int.hydro_type,
                hs_int.hydro_depth_to_greater_flag,
                hs_int.comments,
                s_unit.map_symbol,
                s_unit.strat_name,
                s_unit.hydrostrat_desc,
                hs_subint.hydro_depth_from AS subunit_depth_from,
                hs_subint.hydro_depth_to   AS subunit_depth_to,
                hs_subint.hydro_subunit_code,
                hs_subunit.hydro_subunit_desc,
                hs_subint.hydro_depth_from as depth_from,
                hs_subint.hydro_depth_to as depth_to,
                To_char(s_unit.map_symbol) 
                || '(' 
                || 
                To_char(hs_subint.hydro_subunit_code) || ')' as hsunit_symbol,
                Trim(To_char(s_unit.strat_name)) || ' (' || Trim(To_char(hs_subunit.hydro_subunit_desc)) || ')' as hsunit_desc
FROM            dhdb.wa_hydrostrat_int_vw hs_int
left outer join dhdb.st_strat_unit_vw s_unit
ON              hs_int.strat_unit_no = s_unit.strat_unit_no
left outer join dhdb.wa_hydrostrat_subint_vw hs_subint
ON              hs_int.hydro_int_no = hs_subint.hydro_int_no
left outer join dhdb.wa_hydrostrat_subunit_vw hs_subunit
ON              hs_subint.strat_unit_no = hs_subunit.strat_unit_no
AND             hs_subint.hydro_subunit_code = hs_subunit.hydro_subunit_code
left outer join dhdb.dd_drillhole_vw dh
ON              hs_int.drillhole_no = dh.drillhole_no
left outer join dhdb.dd_drillhole_summary_vw summ
ON              hs_int.drillhole_no = summ.drillhole_no
WHERE           hs_int.drillhole_no IN (280002)
 and hs_subint.hydro_subunit_code is not null

"""